# Clean scrapping by commities and sessions

Цель: получить обсуждение законопроектов в первом чтении для каждого комитета для каждой сессии Думы с 2011 года

## Алгоритм:

1) Вручную указать ссылки на страницы со списком хроникам для каждого созыва (страницы, на которых собраны ссылки).

   Переменная: **session_X_chronicles_pages**
   
2) Вручную указать ссылки на страницы со списком стенограмм для каждого созыва.

   Переменная: **session_X_stenograms_pages**

3) Получить ссылки на все хроники созыва X
   
   Функция: **get_all_chronicles_links**(session_chronicles_pages)   

   Переменная: **session_X_chronicles_links**
   
4) Получить ссылки на все стенограммы созыва X
   
   Функция: **get_all_stenograms_links**(session_stenograms_pages)   

   Переменная: **session_X_stenograms_links** 
   
5) Из каждой хроники получить список пунктов повестки, в которых законопроекты первого чтения имели обсуждение (были заданы вопросы). 

   Функция для одной хроники: **get_discussed_numbers**
    
   Функция для списка ссылок на хроники: **get_all_discussed_numbers**(session_chronicles_links)
    
   Переменная: **session_X_discussed_numbers**

In [72]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from transliterate import translit

In [73]:
list_of_commities = ['аграрным вопросам',
 'безопасности и противодействию коррупции',
 'бюджету и налогам',
 'вопросам семьи, женщин и детей',
 'вопросам собственности, земельным и имущественным отношениям',
 'государственному строительству и законодательству',
 'делам Содружества Независимых Государств, евразийской интеграции и связям с соотечественниками',
 'делам национальностей',
 'защите конкуренции',
 'информационной политике, информационным технологиям и связи',
 'контролю',
 'культуре',
 'малому и среднему предпринимательству',
 'международным делам',
 'молодежной политике',
 'науке и высшему образованию',
 'обороне',
 'охране здоровья',
 'промышленности и торговле',
 'просвещению',
 'развитию Дальнего Востока и Арктики',
 'развитию гражданского общества, вопросам общественных и религиозных объединений',
 'региональной политике и местному самоуправлению',
 'строительству и жилищно-коммунальному хозяйству',
 'транспорту и развитию транспортной инфраструктуры',
 'труду, социальной политике и делам ветеранов',
 'туризму и развитию туристической инфраструктуры',
 'физической культуре и спорту',
 'финансовому рынку',
 'экологии, природным ресурсам и охране окружающей среды',
 'экономической политике',
 'энергетике']

In [74]:
def get_links_chronicles(url):
    '''
    Function will return a list in which 
    every element shows
    [date of session][link to chronicle of session]
    '''
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    raw_chronicles = []
    for link in soup.find_all("a", href = re.compile(r"^/node/\d+/")):
        raw_chronicles.append(str(link))    

    links_chronicles = []
    beginning = 'http://transcript.duma.gov.ru/'
    for i in raw_chronicles:
        link_start = i.find('node')
        link = beginning +  i[link_start:20]
        date_start = i.find('Хроника')
        date_ending = i.find('<', i.find('Хроника'))
        date = i[date_start:date_ending]
        links_chronicles.append([date, link])

    return links_chronicles

In [75]:
def get_links_stenograms(url):
    
    '''    
    На вход: ссылка на страницу со ссылками на стенограмму.
    
    Возвращает: список формата    
    [дата сессии][ссылка на стенограмму]
    
    '''
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    raw_stenograms = []
    for link in soup.find_all("a", href = re.compile(r"^/node/\d+/")):
        raw_stenograms.append(str(link))    

    links_stenograms = []
    beginning = 'http://transcript.duma.gov.ru/'
    for i in raw_stenograms:
        link_start = i.find('node')
        link = beginning +  i[link_start:20]
        date_start = i.find('Стенограмма')
        date_ending = i.find('<', i.find('Стенограмма'))
        date = i[date_start:date_ending]
        links_stenograms.append([date, link])

    return links_stenograms

In [76]:
def get_all_chronicles_links(session_chronicles_pages):    
    
    '''
    На вход: страницы, на которых много ссылкок на хроники.
    Возвращает: список всех ссылок на все эти хроники. 
    '''
    
    session_chronicles_links = []
    for i in session_chronicles_pages:
        chron = get_links_chronicles(i)
        only_links = list(map(lambda x: x[1], chron))
        session_chronicles_links += (only_links)
        
    return session_chronicles_links

In [77]:
def get_all_stenograms_links(session_stenograms_pages):
    
    '''
    На вход: страницы, на которых много ссылкок на стенограммы.
    Возвращает: словарь всех ссылок на все эти стенограммы
    (ключ - дата)
    '''
    
    session_stenograms_links = []

    for i in session_stenograms_pages:
        session_stenograms_links.extend(get_links_stenograms(i))
        
    
    # создадим словарь, чтобы было удобнее обращаться к элементам

    session_stenograms_links_dict = dict()
    for i in session_stenograms_links:
        key = i[0]
        key = key[key.find(' ', key.find('засед')) +1 :]
        value = i[1]
        session_stenograms_links_dict[key] = value
        
    return session_stenograms_links_dict

In [78]:
def get_discussed_numbers(url):
    
    '''
    на вход - ссылка на хронику.
    returns:
    * дата заседания
    * пункт повестки, в котором было обсуждение,
    * индикатор того, был ли содоклад,
    * какой комитет выступал    
    
    '''

    #url = 'http://transcript.duma.gov.ru/node/5969/'

    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    date_start = soup.text.find('Хроника заседания')
    date_end = soup.text.find('.')
    date = soup.text[date_start: date_end+1]    

    start_phrase = 'депутаты приступили к рассмотрению законопроектов в первом чтении'
    if start_phrase is None:
        return None
    start_tag = soup.find(lambda tag: tag.name == 'p' and start_phrase in tag.text)
    if start_tag is None:
        return None
    start_index = soup.find_all('p').index(start_tag)
    
    results = []

    for p in soup.find_all('p')[start_index+1:]:
        phrases = ['ответил на вопросы', 'ответила на вопросы', \
                   'ответили на вопросы', 'на вопросы ответил', \
                   'на вопросы ответила', 'на вопросы ответили',\
                  'прениях']
        if any(phrase in p.text.lower() for phrase in phrases):
            item = []
            item.append(date)
            paragraph = p.text.lower()
            start = paragraph.find('повестки дня')
            #number = paragraph[start-3: start]
            helper_to_find_number = paragraph[start-9: start]
            number = ''
            for el in helper_to_find_number:
                if el.isdigit():
                    number += el
                if el == '.':
                    number = 'no'
                    break
                if el == 'и':
                    break
            item.append(number)
            
            other_speaker = 0
            if 'содоклад' in paragraph:
                other_speaker = 1
            if 'содоклад' not in paragraph:
                if 'выступил' in paragraph and 'изложил' in paragraph:
                    other_speaker = 1
                if 'выступил' in paragraph and paragraph.count('выступил') > 1:
                    other_speaker = 1
                    
            item.append(other_speaker)
            if 'комитета государственной думы по' in paragraph:
                found_options = ['']
                # устранение ошибки по типу: "культуре", "культуре и спорту"
                # возьмем самое длинное (=самое полное) совпадение
                for c in list_of_commities:
                    if c in paragraph: 
                        found_options.append(c)
                item.append(max(found_options))
                        
            if 'комитета государственной думы по' not in paragraph:
                item.append(0)
                
            results.append(item)
          
    results = pd.DataFrame(results)
    results.columns = ['дата', 'пункт повестки', 'наличие содоклада', 'комитет']
    results = results[results['пункт повестки'] != 'no']
            
    return results

In [79]:
#get_discussed_numbers('http://transcript.duma.gov.ru/node/5455/')

In [80]:
def get_all_discussed_numbers(session_chronicles_links):
    
    '''
    На вход: ссылки на все хроники
    На выход: список указаний на то, в какую дату какой пункт повестки
    имел обсуждение (были заданы вопросы), был ли представлен содоклад,
    к какому комитету относится обсуждение
    
    '''
    
    session_discussed_numbers = []
    
    for i in session_chronicles_links:
        discussions = get_discussed_numbers(i)
        try:
            if len(discussions) > 0:
                session_discussed_numbers.append(get_discussed_numbers(i))
            else:
                continue
        except TypeError as te:
            continue
        
    session_discussed_numbers = [i for i in session_discussed_numbers if i is not None]
    
    session_discussed_numbers = pd.concat(session_discussed_numbers)
        
    return session_discussed_numbers    

In [81]:
# словарь с соответствие названия коммитета и транслита первых пяти букв

dict_translits = dict()
for i in list_of_commities:
    translited = translit(i, 'ru', reversed=True)
    if 'voprosam' in translited:
        translited = translited[translited.find(' ') + 1:]
    translited = translited[:5]
    translited = translited.replace("'", '')
    dict_translits[i] = translited
dict_translits

{'аграрным вопросам': 'vopro',
 'безопасности и противодействию коррупции': 'bezop',
 'бюджету и налогам': 'bjudz',
 'вопросам семьи, женщин и детей': 'semi',
 'вопросам собственности, земельным и имущественным отношениям': 'sobst',
 'государственному строительству и законодательству': 'gosud',
 'делам Содружества Независимых Государств, евразийской интеграции и связям с соотечественниками': 'delam',
 'делам национальностей': 'delam',
 'защите конкуренции': 'zasch',
 'информационной политике, информационным технологиям и связи': 'infor',
 'контролю': 'kontr',
 'культуре': 'kult',
 'малому и среднему предпринимательству': 'malom',
 'международным делам': 'mezhd',
 'молодежной политике': 'molod',
 'науке и высшему образованию': 'nauke',
 'обороне': 'oboro',
 'охране здоровья': 'ohran',
 'промышленности и торговле': 'promy',
 'просвещению': 'prosv',
 'развитию Дальнего Востока и Арктики': 'razvi',
 'развитию гражданского общества, вопросам общественных и религиозных объединений': 'grazh',

In [82]:
def get_variables_links_by_commities(session_discussed_numbers, n):
    
    '''
    Функция создает переменные со всеми датами обсужденных вопросов
    для каждого комитета отдельно
    
    Возвращает список из созданных переменных
    '''    
    session_dialogies_by_commities = []
    
    
    n = str(n)
    for i in set(session_discussed_numbers['комитет']):
        if i not in [0, None, '']:
            
            translited = translit(i, 'ru', reversed=True)
            if 'voprosam' in translited:
                translited = translited[translited.find(' ') + 1:]
            translited = translited[:5]
            name = 'session_' + n + '_dialog_commitee_'+ str(translited)
            name = name.replace("'", '')
            globals()[name] = session_discussed_numbers[session_discussed_numbers['комитет'] == i]
            session_dialogies_by_commities.append(name)
        if i == None or i == '' or i == 0:
            continue  
             
    return session_dialogies_by_commities

In [109]:
def get_clean_transcript(url):
    
    '''
    На вход: ссылка на стенограмму
    На выход: транскрипт стенограммы в формате 
    [кто говорит, что говорит]
    
    '''
    
    #url = "http://transcript.duma.gov.ru/node/5750/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    try:
        soup.i.decompose()
    # здесь возможно нужен счетчик ошибок, чтобы не было stackoverflow error 
    except AttributeError as e:
        print('I encountered website error, rerunning the function now, please wait')
        get_clean_transcript(url)
        
    discussion = []

    who = ''
    what = ''
    for i in soup.find_all(['p']):
        if i.find(['b']):
            if len(who) > 0:
                if len(what) != 0:
                    discussion.append([who, what])
                if len(what) == 0:
                    continue
            who = (i.find(['b']).text).replace(',', '')
            what = (i.text).replace(i.find(['b']).text, '')
            if 'фракц' in what or 'официальный представитель' in what:
                what = ''
        else:
            what += i.text

    return discussion

In [84]:
def get_certain_discussion(url, number, additional_speech):
    
    '''
    Функция возвращает сессию вопросов и ответов по определенному пункту повестки дня
    На вход: ссылка, номер повестки, индикатор наличия содоклада
    На выход: 
        список: за вопросом следует ответ
    
    '''
    number = str(number)
    #url = "http://transcript.duma.gov.ru/node/5750/"
    corpus = get_clean_transcript(url)
    
    markers = [number + '-й вопрос',
    'пункт ' + number,
    number + '-й пункт',
    'рассмотрению вопроса ' + number,
    number + '-й, проект',
    number + '-го вопроса',
    number + ' вопроса',
    number + '-му вопросу',
    'вопрос ' + number,
    number + ' вопрос',
    'пункту ' + number ]

    
    number_mentions = []
    
    # найти начало обсуждения определенного пункта
    for index, value in enumerate(corpus):
        if any(marker in (value[1]).lower() for marker in markers)\
            and value[0] == 'Председательствующий.': # только он объявляет повестку
                number_mentions.append(index)
                #просьба рассмотреть вне очереди
        if any(marker in (value[1]).lower() for marker in markers) \
            and ('просьб' in value[1] or 'отклон' in value[1])\
            and value[0] != 'Председательствующий.':
            number_mentions.append(index+1)
        if 'снять' in value[1]:
            return []
                
        
    if len(number_mentions) > 0:
        start = number_mentions[0]
    if len(number_mentions) == 0: # ведущий не говорил четко о начале конкретного пункта
        return []
    
    # найти конец обсуждения
    for i in range(start+1, len(corpus)):
        if 'желающие выступить' in corpus[i][1].lower():
            end = i
            break
        if "результаты голосования" in corpus[i][1].lower():
            end = i
            break
        if "покажите результаты" in corpus[i][1].lower():
            end = i
            break
    
    # вырезать диалог без дополнительных комментариев
    dialog = corpus[start:end]
    dialog[:] = [i for i in dialog if i[0] != 'Председательствующий.']
    dialog[:] = [i for i in dialog if i[0] != 'Из зала.']
    
    # склеить реплики одного и того же человека (могли быть прерваны председателем)
    for i in range(1, len(dialog)):
        if dialog[i][0] == dialog[i-1][0]:
            name  = dialog[i][0]
            content = dialog[i-1][1] + dialog[i][1]
            dialog[i-1] = [] # заменим пустым списком, чтобы не изменять длину диалога
            dialog[i] = [name, content]
            
    # убрать пустые элементы
    dialog[:] = [i for i in dialog if len(i) > 0]
    
    
    person1 = dialog[0][0]
    if additional_speech == 1:
        try:
            person2 = dialog[1][0]
        except IndexError as ie:
            return []
    
    # убрать выступление докладчика и содокладчика, интересуют только вопросы и ответы
    
    dialog[:] = dialog[1:]
    if additional_speech == 1:   
        dialog[:] = dialog[1:]
        
        speakers = [person1, person2]
        
        # склеить ответы докладчика и содокладчика, если они рядом 
        # (отвечают на один и тот же вопрос)
        
        for i in range(1, len(dialog)):

            if dialog[i][0] in speakers and dialog[i-1][0] in speakers: 
                name  = 'оба спикера'
                content = dialog[i-1][1] + dialog[i][1]
                dialog[i-1] = [] # заменим пустым списком, чтобы не изменять длину диалога
                dialog[i] = [name, content]
            
    # убрать пустые элементы
    dialog[:] = [i for i in dialog if len(i) > 0]       
    
    # оставить только реплики без указания на то, кто говорит
    dialog[:] = [i[1] for i in dialog]
    
    return dialog

In [100]:
def get_all_clean_dialogues_by_commitee(session_dialog_commitee, session_stenograms_links_dict):
    
    '''
    На вход: переменная со списком, указывающим на даты заседения и
    для имеющих обсуждение пунктов повестки
    
    На выход: список с последовательными сессиями вопросов и ответов
    для определенного коммитета
    '''
    session_all_dialogs = []

    for row_chron in session_dialog_commitee.values.tolist():
        chron_date = row_chron[0]
        chron_date = chron_date[chron_date.find(' ', chron_date.find('заседания')) +1:]
        question_number = row_chron[1]
        indication = row_chron[2]
    
        try:
            link_stenogram = session_stenograms_links_dict[chron_date]
        except KeyError as ke:
            #print('I encountered internal error, rerunnig the fucntion now')
            #link_stenogram = session_stenograms_links_dict[chron_date]
            continue
        #print(link_stenogram, int(question_number), indication)

        certain_discussion = get_certain_discussion(link_stenogram, int(question_number), indication)
        if len(certain_discussion) == 0:
            continue
            
        # на сайте госдумы не везде правильная разметка, какие-то речи не выделены шрифтом
        # поэтому такие случаи отловим вручную
        if len(certain_discussion) % 2 == 0 and len(certain_discussion) != 0 :
            session_all_dialogs.extend(certain_discussion)
    
    return session_all_dialogs

In [86]:
def get_certain_discussion_new(url, number, additional_speech):
    number = str(number)
    #url = "http://transcript.duma.gov.ru/node/5735/"
    corpus = get_clean_transcript(url)

    markers = [number + '-й вопрос',
        'пункт ' + number,
        number + '-й пункт',
        'рассмотрению вопроса ' + number,
        number + '-й, проект',
        number + '-го вопроса',
        number + ' вопроса',
        number + '-му вопросу',
        'вопрос ' + number,
        number + ' вопрос',
        'пункту ' + number ]


    number_mentions = []

        # найти начало обсуждения определенного пункта
    for index, value in enumerate(corpus):
        speech = value[1].split()
        for m in markers:
            if m in speech and str(number) in speech\
            and value[0] == 'Председательствующий.': # только он объявляет повестку
                number_mentions.append(index)
                    #просьба рассмотреть вне очереди

            if m in speech and str(number) in speech \
            and 'просьб' in value[1] \
            and value[0] != 'Председательствующий.':
                number_mentions.append(index+1)
        if 'снять' in value[1]:
            return []


    if len(number_mentions) > 0:
        start = number_mentions[0]
    if len(number_mentions) == 0: # ведущий не говорил четко о начале конкретного пункта
        return []

        # найти конец обсуждения
    for i in range(start+1, len(corpus)):
        if 'желающие выступить' in corpus[i][1].lower():
            end = i
            break
        if "результаты голосования" in corpus[i][1].lower():
            end = i
            break
        if "покажите результаты" in corpus[i][1].lower():
            end = i
            break

        # вырезать диалог без дополнительных комментариев
    dialog = corpus[start:end]
    dialog[:] = [i for i in dialog if i[0] != 'Председательствующий.']
    dialog[:] = [i for i in dialog if i[0] != 'Из зала.']

        # склеить реплики одного и того же человека (могли быть прерваны председателем)
    for i in range(1, len(dialog)):
        if dialog[i][0] == dialog[i-1][0]:
            name  = dialog[i][0]
            content = dialog[i-1][1] + dialog[i][1]
            dialog[i-1] = [] # заменим пустым списком, чтобы не изменять длину диалога
            dialog[i] = [name, content]

        # убрать пустые элементы
    dialog[:] = [i for i in dialog if len(i) > 0]


    person1 = dialog[0][0]
    if additional_speech == 1:
        person2 = dialog[1][0] 

        # убрать выступление докладчика и содокладчика, интересуют только вопросы и ответы

    dialog[:] = dialog[1:]
    if additional_speech == 1:   
        dialog[:] = dialog[1:]

        speakers = [person1, person2]

            # склеить ответы докладчика и содокладчика, если они рядом 
            # (отвечают на один и тот же вопрос)

        for i in range(1, len(dialog)):

            if dialog[i][0] in speakers and dialog[i-1][0] in speakers: 
                name  = 'оба спикера'
                content = dialog[i-1][1] + dialog[i][1]
                dialog[i-1] = [] # заменим пустым списком, чтобы не изменять длину диалога
                dialog[i] = [name, content]

        # убрать пустые элементы
    dialog[:] = [i for i in dialog if len(i) > 0]       

        # оставить только реплики без указания на то, кто говорит
    dialog[:] = [i[1] for i in dialog]

    return dialog

# Сбор данных

In [87]:
# Ссылки на страницы с стенограмм всех сессий 7 созыва:

session_7_stenograms_pages = [\
'http://transcript.duma.gov.ru/search/?sessid=4519&doctype=3&dt_start=&dt_end=&phrase1=',
'http://transcript.duma.gov.ru/search/?sessid=4573&doctype=3&dt_start=&dt_end=&phrase1=',
'http://transcript.duma.gov.ru/search/?by=date&sessid=4573&doctype=3&dt_start=&dt_end=&phrase1=&PAGEN_1=2',
'http://transcript.duma.gov.ru/search/?sessid=4716&doctype=3&dt_start=&dt_end=&phrase1=',
'http://transcript.duma.gov.ru/search/?sessid=4806&doctype=3&dt_start=&dt_end=&phrase1=',
'http://transcript.duma.gov.ru/search/?by=date&sessid=4806&doctype=3&dt_start=&dt_end=&phrase1=&PAGEN_1=2',
'http://transcript.duma.gov.ru/search/?sessid=4961&doctype=3&dt_start=&dt_end=&phrase1=',
'http://transcript.duma.gov.ru/search/?by=date&sessid=4961&doctype=3&dt_start=&dt_end=&phrase1=&PAGEN_1=2',
'http://transcript.duma.gov.ru/search/?sessid=5081&doctype=3&dt_start=&dt_end=&phrase1=',
'http://transcript.duma.gov.ru/search/?by=date&sessid=5081&doctype=3&dt_start=&dt_end=&phrase1=&PAGEN_1=2',
'http://transcript.duma.gov.ru/search/?sessid=5256&doctype=3&dt_start=&dt_end=&phrase1=',
'http://transcript.duma.gov.ru/search/?by=date&sessid=5256&doctype=3&dt_start=&dt_end=&phrase1=&PAGEN_1=2',
 'http://transcript.duma.gov.ru/search/?sessid=5368&doctype=3&dt_start=&dt_end=&phrase1=',
'http://transcript.duma.gov.ru/search/?by=date&sessid=5368&doctype=3&dt_start=&dt_end=&phrase1=&PAGEN_1=2',
'http://transcript.duma.gov.ru/search/?sessid=5503&doctype=3&dt_start=&dt_end=&phrase1=',
'http://transcript.duma.gov.ru/search/?sessid=5579&doctype=3&dt_start=&dt_end=&phrase1=',
 'http://transcript.duma.gov.ru/search/?by=date&sessid=5579&doctype=3&dt_start=&dt_end=&phrase1=&PAGEN_1=2' ]  
    
    
# Ссылки на страницы с хрониками всех сессий 7 созыва:

session_7_chronicles_pages = \
['http://transcript.duma.gov.ru/search/?sessid=4519&doctype=2&dt_start=&dt_end=&phrase1=',
'http://transcript.duma.gov.ru/search/?sessid=4573&doctype=2&dt_start=&dt_end=&phrase1=',
'http://transcript.duma.gov.ru/search/?by=date&sessid=4573&doctype=2&dt_start=&dt_end=&phrase1=&PAGEN_1=2',
 'http://transcript.duma.gov.ru/search/?sessid=4573&doctype=2&dt_start=&dt_end=&phrase1=',
 'http://transcript.duma.gov.ru/search/?by=date&sessid=4573&doctype=2&dt_start=&dt_end=&phrase1=&PAGEN_1=2',
 'http://transcript.duma.gov.ru/search/?sessid=4806&doctype=2&dt_start=&dt_end=&phrase1=',
'http://transcript.duma.gov.ru/search/?by=date&sessid=4806&doctype=2&dt_start=&dt_end=&phrase1=&PAGEN_1=2',
 'http://transcript.duma.gov.ru/search/?sessid=4961&doctype=2&dt_start=&dt_end=&phrase1=',
 'http://transcript.duma.gov.ru/search/?by=date&sessid=4961&doctype=2&dt_start=&dt_end=&phrase1=&PAGEN_1=2',
 'http://transcript.duma.gov.ru/search/?sessid=5081&doctype=2&dt_start=&dt_end=&phrase1=',
 'http://transcript.duma.gov.ru/search/?by=date&sessid=5081&doctype=2&dt_start=&dt_end=&phrase1=&PAGEN_1=2',
 'http://transcript.duma.gov.ru/search/?sessid=5256&doctype=2&dt_start=&dt_end=&phrase1=',
 'http://transcript.duma.gov.ru/search/?by=date&sessid=5256&doctype=2&dt_start=&dt_end=&phrase1=&PAGEN_1=2',
 'http://transcript.duma.gov.ru/search/?sessid=5368&doctype=2&dt_start=&dt_end=&phrase1=',
 'http://transcript.duma.gov.ru/search/?by=date&sessid=5368&doctype=2&dt_start=&dt_end=&phrase1=&PAGEN_1=2',
'http://transcript.duma.gov.ru/search/?sessid=5503&doctype=2&dt_start=&dt_end=&phrase1=',
'http://transcript.duma.gov.ru/search/?sessid=5579&doctype=2&dt_start=&dt_end=&phrase1=',
 'http://transcript.duma.gov.ru/search/?by=date&sessid=5579&doctype=2&dt_start=&dt_end=&phrase1=&PAGEN_1=2'
]


In [110]:
session_7_chronicles_links = get_all_chronicles_links(session_7_chronicles_pages)

In [111]:
session_7_stenograms_links_dict = get_all_stenograms_links(session_7_stenograms_pages)

In [112]:
session_7_discussed_numbers = get_all_discussed_numbers(session_7_chronicles_links)

In [113]:
session_7_dialogies_by_commities = \
    get_variables_links_by_commities(session_7_discussed_numbers, 7)

session_7_dialogies_by_commities

['session_7_dialog_commitee_bezop',
 'session_7_dialog_commitee_gosud',
 'session_7_dialog_commitee_semi',
 'session_7_dialog_commitee_oboro',
 'session_7_dialog_commitee_ohran',
 'session_7_dialog_commitee_ekono',
 'session_7_dialog_commitee_delam',
 'session_7_dialog_commitee_kontr',
 'session_7_dialog_commitee_grazh',
 'session_7_dialog_commitee_kult',
 'session_7_dialog_commitee_bjudz',
 'session_7_dialog_commitee_finan',
 'session_7_dialog_commitee_trudu',
 'session_7_dialog_commitee_vopro',
 'session_7_dialog_commitee_infor',
 'session_7_dialog_commitee_energ']

In [114]:
session_7_ready_dialogs = []
for comm in session_7_dialogies_by_commities:
    indication_comm = comm[len(comm) - comm[::-1].find('_'):]
    comm = eval(comm)
    variable_name = 'session_7_all_dialogs_'+ indication_comm
    print(variable_name)
    session_7_ready_dialogs.append(variable_name)
    
    # создание самой переменной
    globals()[variable_name] = get_all_clean_dialogues_by_commitee(\
       comm, session_7_stenograms_links_dict)

session_7_all_dialogs_bezop
session_7_all_dialogs_gosud
session_7_all_dialogs_semi
session_7_all_dialogs_oboro
session_7_all_dialogs_ohran
session_7_all_dialogs_ekono
session_7_all_dialogs_delam
session_7_all_dialogs_kontr
session_7_all_dialogs_grazh
session_7_all_dialogs_kult
session_7_all_dialogs_bjudz
session_7_all_dialogs_finan
session_7_all_dialogs_trudu
session_7_all_dialogs_vopro
session_7_all_dialogs_infor
session_7_all_dialogs_energ


In [115]:
session_7_all_dialogs_oboro

['У меня короткий вопрос. У меня огромное количество друзей-военнослужащих, и вот им иногда предлагают жильё не там, где они хотят, а вот там, где они хотят, либо нет жилья таких размеров, либо его вообще нет — вот как-то законопроект поможет решению этого или он по другой части, по технической части?',
 ' Сергей Алексеевич, спасибо за вопрос. В одной из новелл в этом законопроекте речь идёт о предоставлении военнослужащим жилья не только равной или большей площади, но и меньшей площади, но должно быть письменное согласие всех совершеннолетних членов семьи военнослужащего. Если такое согласие не будет получено, то военнослужащий ждёт квартиру той площади, которая ему положена.']

In [117]:
#работающий код на выгрузку готовых диалогов в формате txt

for ready_dialog in session_7_ready_dialogs:
    dialog = eval(ready_dialog)
    with open(ready_dialog +'.txt', 'w', encoding='utf-8') as f:
        for item in dialog:
            f.write("%s\n" % item)
        